In [1]:
import os
from src.main import ClusterService
os.chdir('..')

/home/vpavlishen/.virtualenvs/face-recognition/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/vpavlishen/.virtualenvs/face-recognition/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/vpavlishen/.virtualenvs/face-recognition/l

# ClusterService

This class is used for clustering the images and evaluation results

In [2]:
service = ClusterService(path_to_images='data/processed/test-task/clusters', path_to_target_clusters='data/processed/test-task/clusters.csv')

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/vpavlishen/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/vpavlishen/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/vpavlishen/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /home/vpavlishen/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/vpavlishen/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


Target clusters contain different cluster_id with background images. But model for generating embeddings of faces ignore background images. That's why full evaluation according to 'clusters.csv' is poor. However, after face filtering images and clustering them, quality of algorithm dramatically improved 

In [3]:
service.run()


Background and face images clustering: 
Homogeneity score: 0.3965393867896544
Completeness score: 0.6129324424069272
V-Measure score: 0.4815426202610689

Face images clustering: 
Homogeneity score: 0.9768697821391191
Completeness score: 0.8898553170854854
V-Measure score: 0.9313345281506219
